In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("cryptoData") \
.config("spark.driver.bindAddress", "0.0.0.0") \
.getOrCreate()

In [3]:
stock_symbols = [
    'AAPL', 'MSFT', 'NVDA', 'META', 'AMZN', 'TSLA', 'GOOGL',
    'DBD', 'DSGX', 'GTLB', 'LOGI', 'CRSR',
    'LNG', 'SWN', 'APA', 'BTU', 'CL',
    'BMY', 'THC', 'TNDM',
    'MOS', 'AXTA', 'KOP',
    'SBLK', 'EME', 'DNOW',
]

In [4]:
len(stock_symbols)

26

In [5]:
data = {}
for i in stock_symbols:
    df = yf.download(i)
    df = df.reset_index()
    data[i] = df


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
len(data.keys())

26

In [7]:
url = "jdbc:postgresql://localhost:5432/postgres"


In [8]:
data["NVDA"].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-01-22,0.437500,0.488281,0.388021,0.410156,0.376255,271468800
1,1999-01-25,0.442708,0.458333,0.410156,0.453125,0.415672,51048000
2,1999-01-26,0.458333,0.467448,0.411458,0.417969,0.383422,34320000
3,1999-01-27,0.419271,0.429688,0.395833,0.416667,0.382228,24436800
4,1999-01-28,0.416667,0.419271,0.412760,0.415365,0.381033,22752000


In [9]:
spark_df = spark.createDataFrame(data['NVDA'])

In [10]:
spark_df.show(n=3)

+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+---------+
|               Date|               Open|               High|                Low|             Close|          Adj Close|   Volume|
+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+---------+
|1999-01-22 00:00:00|             0.4375| 0.4882810115814209|0.38802099227905273|0.4101560115814209| 0.3762546479701996|271468800|
|1999-01-25 00:00:00|0.44270798563957214|0.45833298563957214| 0.4101560115814209|          0.453125|0.41567203402519226| 51048000|
|1999-01-26 00:00:00|0.45833298563957214|0.46744799613952637|0.41145798563957214|0.4179689884185791| 0.3834218382835388| 34320000|
+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+---------+
only showing top 3 rows



In [11]:
properties = {
    "username":"postgres",
    "password":"password",
    "driver":"org.postgresql.Driver"
}

In [12]:
# spark.stop()

In [13]:
processed_Data = {}
for i in stock_symbols:
    print(i)
    df = spark.createDataFrame(data[i])
    selected_df = df.select("Date","Close","Volume")
    selected_df = selected_df.withColumnRenamed("Close",i)
    selected_df = selected_df.withColumnRenamed("Volume",f'{i}_vol')
    processed_Data[i] = selected_df
    


AAPL
MSFT
NVDA
META
AMZN
TSLA
GOOGL
DBD
DSGX
GTLB
LOGI
CRSR
LNG
SWN
APA
BTU
CL
BMY
THC
TNDM
MOS
AXTA
KOP
SBLK
EME
DNOW


In [109]:
processed_Data["AAPL"].show()

+-------------------+-------------------+---------+
|               Date|               AAPL| AAPL_vol|
+-------------------+-------------------+---------+
|1980-12-12 00:00:00| 0.1283479928970337|469033600|
|1980-12-15 00:00:00|0.12165199965238571|175884800|
|1980-12-16 00:00:00|0.11272300034761429|105728000|
|1980-12-17 00:00:00|0.11551299691200256| 86441600|
|1980-12-18 00:00:00|0.11886200308799744| 73449600|
|1980-12-19 00:00:00|0.12611599266529083| 48630400|
|1980-12-22 00:00:00| 0.1322540044784546| 37363200|
|1980-12-23 00:00:00|0.13783499598503113| 46950400|
|1980-12-24 00:00:00|0.14508900046348572| 48003200|
|1980-12-26 00:00:00|0.15848200023174286| 55574400|
|1980-12-29 00:00:00|0.16071400046348572| 93161600|
|1980-12-30 00:00:00|  0.156808003783226| 68880000|
|1980-12-31 00:00:00| 0.1523440033197403| 35750400|
|1981-01-02 00:00:00|0.15401799976825714| 21660800|
|1981-01-05 00:00:00|0.15067000687122345| 35728000|
|1981-01-06 00:00:00| 0.1439729928970337| 45158400|
|1981-01-07 

In [ ]:
for i in stock_symbols:
    print(len(data[i]))

10892
9566
6315
2962
6740
3439
4914
136
6312
595
6774
862
7528
11080
11292
1737
12817
15645
15645
13046
11080
2587
9093
2337
4548
4086
7334
2460


In [18]:
stock_symbols[7], len(data[stock_symbols[7]])

('DBD', 136)

In [20]:
df = spark.createDataFrame(data["DBD"])

In [21]:
df.show()

+------------------+------------------+------------------+------------------+------------------+------+
|              Open|              High|               Low|             Close|         Adj Close|Volume|
+------------------+------------------+------------------+------------------+------------------+------+
|              24.0|  25.3799991607666|18.565000534057617| 20.56999969482422| 20.56999969482422|485400|
|19.799999237060547|              21.5|19.399999618530273|              21.5|              21.5|137600|
|              20.5|              20.5|  17.8799991607666|19.440000534057617|19.440000534057617|250200|
|              19.0|19.760000228881836| 18.76799964904785|19.290000915527344|19.290000915527344|103900|
|18.600000381469727| 20.34000015258789|18.600000381469727| 20.06999969482422| 20.06999969482422|378500|
| 20.59000015258789|              21.0|19.959999084472656|20.190000534057617|20.190000534057617| 32300|
|20.229999542236328|20.389999389648438|19.920000076293945|20.100